In [1]:
import sys,os
sys.path.append("/home/paco/Documents/site_similarity")

In [2]:
from utils.notebook_utils import load_level_data, create_nodes, create_graph, draw_graph

In [3]:
lvl_one = load_level_data(data_path='/home/paco/Documents/site_similarity/notebooks/clean_data_20200808.json', level=1)  

Loaded 3489 nodes with records level <= 1 and child size:16981


In [4]:
lvl_one_nodes = create_nodes(lvl_one)

In [5]:
lvl_one_nodes[:10]

[('crooked.com', 'votesaveamerica.com'),
 ('crooked.com', 'art19.com'),
 ('crooked.com', 'promocodeportal.com'),
 ('crooked.com', 'mediamatters.org'),
 ('crooked.com', 'actblue.com'),
 ('deepleftfield.info', 'politicalflare.com'),
 ('deepleftfield.info', 'bluedotdaily.com'),
 ('antifascistnews.net', 'angrywhitemen.org'),
 ('antifascistnews.net', 'itsgoingdown.org'),
 ('antifascistnews.net', 'torchantifa.org')]

In [6]:
[(k, v) for k, v in lvl_one_nodes if k == 'politicalmayhem.news']

[('politicalmayhem.news', 'politicalmayhem.news')]

In [6]:
len(lvl_one)

3489

In [7]:
from stellargraph import StellarGraph
import pandas as pd


lvl_one_graph = StellarGraph(edges=pd.DataFrame(lvl_one_nodes, columns=['source', 'target']))

In [8]:
from stellargraph.data import BiasedRandomWalk

rw = BiasedRandomWalk(lvl_one_graph)

walks = rw.run(
    nodes=list(lvl_one_graph.nodes()),  # root nodes
    length=100,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
)
print("Number of random walks: {}".format(len(walks)))

Number of random walks: 101610


In [9]:
from gensim.models import Word2Vec

str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, size=128, window=5, min_count=0, sg=1, workers=2, iter=1)

model.save("node2vec_lvl_one_new_corpus.model")

In [3]:
# Load model

from gensim.models import Word2Vec

model = Word2Vec.load("node2vec_lvl_one_new_corpus.model")

In [5]:
model.wv['crooked.com'].shape

(128,)

In [4]:
from dataprep.load_annotated_data import apply_splits
from utils.notebook_utils import load_corpus
from modelling.baselines import eval_model
import pandas as pd

DATA = load_corpus('new_corpus_2020.csv')
SPLITS = apply_splits(DATA, 'modified_splits_new_corpus_2020.json')

print(SPLITS.keys())

train = pd.DataFrame(SPLITS['train-0'])

test = pd.DataFrame(SPLITS['test-0'])

dict_keys(['test-0', 'train-0', 'test-1', 'train-1', 'test-2', 'train-2', 'test-3', 'train-3', 'test-4', 'train-4'])


In [5]:
X_train = train.apply(lambda x: model.wv[x['source_url_processed']], axis=1)

y_train = train.apply(lambda x: x['fact'], axis=1)

In [6]:
X_test = test.apply(lambda x: model.wv[x['source_url_processed']], axis=1)

y_test = test.apply(lambda x: x['fact'], axis=1)

In [7]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

clf = LogisticRegressionCV(
    Cs=10, cv=5, scoring="accuracy", verbose=True, multi_class="ovr", max_iter=300, random_state=42
)
clf.fit(X_train.values.tolist(), y_train.values.tolist())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    3.7s finished


LogisticRegressionCV(cv=5, max_iter=300, multi_class='ovr', random_state=42,
                     scoring='accuracy', verbose=True)

In [8]:
y_pred = clf.predict(X_test.values.tolist())

accuracy_score(y_test, y_pred)

0.5813953488372093

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=42)
clf.fit(X_train.tolist(), y_train.tolist())

GradientBoostingClassifier(random_state=42)

In [12]:
tree_predict = clf.predict(X_test.tolist())
accuracy_score(y_test, tree_predict)

0.7034883720930233

In [13]:
from sklearn import svm

svm_clf = svm.SVC(decision_function_shape='ovo')

svm_clf.fit(X_train.tolist(), y_train.tolist())

SVC(decision_function_shape='ovo')

In [14]:
svm_predict = svm_clf.predict(X_test.tolist())
accuracy_score(y_test, svm_predict)

0.6511627906976745

In [15]:
lin_clf = svm.LinearSVC() # one-vs-rest
lin_clf.fit(X_train.tolist(), y_train.tolist())

LinearSVC()

In [16]:
lin_predict = lin_clf.predict(X_test.tolist())
accuracy_score(y_test, lin_predict)

0.6104651162790697